In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import seaborn as sb
from sklearn.metrics.pairwise import euclidean_distances

class SOM:
    def __init__(self, map_size, lr=0.1):
        self.map = np.random.random(size=(map_size[0], map_size[1], map_size[2]))
        
        self.lr0 = lr
        self.lr = self.lr0
        
        self.R0 = map_size[0]//6
        self.R = self.R0
        
        self.elements = []
        for w in range(self.map.shape[0]):
            for h in range(self.map.shape[1]):
                self.elements.append([w,h])

        
    def train(self, X, y, T=1000):
        changes = []
        puritylist = []
        for t in range(T):
            prev_map = self.map
            shuffle_ind = np.random.randint(0, len(X), len(X))
            for i in range(len(X)):
                x = X[shuffle_ind[i],:]
                
                winner = self.find_winner(x)
                n_strength = self.get_neigh_strength(winner)
                
                self.update_weights(x, n_strength, len(X))
            
            self.lr = self.lr0 * (1-t/T)
            self.R = self.R0 * (1-t/T)
            
            changes.append(np.linalg.norm(prev_map - self.map))
            puritylist.append(self.calc_purity(X,y))

            if t%10 == 0:
                print('iter: ', t)
        
        return changes, puritylist
    
    def calc_purity(self, X, y):
      self.scores = np.zeros(shape=(self.map.shape[0], self.map.shape[1] ,12))
      for i in range(len(X)):
        x = X[i, :]
        winner = self.find_winner(x)
        iw, jw = winner[0], winner[1]
        for c in range(12):
          if y[i]==c:
            self.scores[iw, jw, c+1] += 1
      purity = np.max(self.scores.reshape(self.scores.shape[0]*self.scores.shape[1],12), axis=-1)
      purity = np.sum(purity)/len(X)
      
      return purity

    def visualize_umat(self):
      self.umat = np.zeros(shape=(self.map.shape[0], self.map.shape[1]))
      for w in range(self.map.shape[0]):
        for h in range(self.map.shape[1]):
          neighs = []
          if w - 1 >=0:
              neighs.append(self.map[w-1,h])
          if w + 1 < self.map.shape[0]:
              neighs.append(self.map[w+1,h])
          if h - 1 >=0:
              neighs.append(self.map[w,h-1])
          if h + 1 < self.map.shape[1]:
              neighs.append(self.map[w,h+1])
          neighs = np.array(neighs)
          self.umat[w,h] = np.sum(euclidean_distances([self.map[w,h]],neighs))
      data_in_cell = np.sum(self.scores.reshape(self.scores.shape[0]*self.scores.shape[1],12), axis=-1).reshape(self.scores.shape[0],self.scores.shape[1])
      plt.title('U-Matrix')
      plt.matshow(self.umat,cmap='gray')
      for (i, j), z in np.ndenumerate(data_in_cell):
        plt.text(j, i, z, ha='center', va='center', c='blue')
      plt.show()

    def visualize_som(self, X, y):        
        self.sommat = np.argmax(self.scores.reshape(self.scores.shape[0]*self.scores.shape[1],12), axis=-1).reshape(
            self.scores.shape[0],self.scores.shape[1])-1
        sb.heatmap(self.sommat, vmin=-1, vmax=10, annot=True)
        plt.show()
    
    def find_winner(self, x):
        rep_x = np.tile(x, [self.map.shape[0], self.map.shape[1], 1])
        dists = np.sum((rep_x - self.map)**2, axis=2)
        winner = np.unravel_index(np.argmin(dists, axis=None), dists.shape)
        
        return winner
    
    def get_neigh_strength(self, winner):
        n_strength = np.zeros(shape=(self.map.shape[0], self.map.shape[1]))
        n_strength = euclidean_distances(np.expand_dims(winner, axis=0), self.elements)**2
        n_strength = np.exp(-n_strength/(2*(self.R**2)))
        n_strength = n_strength.reshape((self.map.shape[0], self.map.shape[1]))
        
        return n_strength
    
    def update_weights(self, x, n_strength, X_len):
        delta = -self.map + x
        self.map = self.map + (self.lr) * np.multiply(delta.T, n_strength.T).T
        
    def extract_features(self, x):
        rep_x = np.tile(x, [self.map.shape[0], self.map.shape[1], 1])
        dists = np.sum((rep_x - self.map)**2, axis=2)
        
        return 1/(1+dists)